# get predictions from final models

In [1]:
import gc
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense, Dropout
from keras import optimizers, regularizers, losses

from keras.models import Model
from keras import backend as K
from keras.callbacks import CSVLogger
from keras import metrics

import scipy
import datetime 

import h5py
import keras

import sys

from sklearn.model_selection import ParameterGrid
import pickle


import os
sys.path.append("/home/nbbwang/bigdrive/AD_Project/")
from IntegratedGradients import *

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

import matplotlib
matplotlib.rcParams.update({'font.size': 18, 'font.family': "Times New Roman"})
plt.rcParams["font.family"] = "Times"


import IntegratedGradients as IG
import keras 

from scipy import stats 


path_to_configs = "../"
sys.path.append(path_to_configs)
from configs import *
from models import *

In [3]:
hy_dict_list = list(ParameterGrid(hyperparams))
MTL_final_final_model = pickle.load(open(path_to_configs+path_to_final_models_chosen + "MTL/final.p", "rb" ) )
baselines_final_final_model = pickle.load(open(path_to_configs+path_to_final_models_chosen + "MLP_baselines/final.p", "rb" ))


In [4]:
phens = ["CERAD", "BRAAK", "PLAQUES", "TANGLES", "ABETA_IHC", "TAU_IHC"]


In [5]:
for dpath in ["", "_intersection"]:
    for dset in os.listdir("%spredictions%s/"%(path_to_ext_val_predictions, dpath)):
        if dpath=="_intersection":
            dsets_to_check = ["train","test"]
        else:
            dsets_to_check = ["test"]

            
        ######### MD-AD AVERAGING ################
        for dset_to_check in dsets_to_check:

            all_pred_vals = []
            if dset_to_check == "train":
                path_to_preds = "%spredictions%s/%s/MTL/%s/train/"%(path_to_ext_val_predictions, dpath,dset, MTL_final_final_model)
            else:
                path_to_preds = "%spredictions%s/%s/MTL/%s/"%(path_to_ext_val_predictions, dpath,dset, MTL_final_final_model)
            print("Saving consensus %s predictions to %s"%(dset_to_check,path_to_preds))

            for f in os.listdir(path_to_preds):
                if os.path.isdir(path_to_preds + f):
                    continue

                pred_df = pd.read_csv("%s/%s"%(path_to_preds,f), index_col="Unnamed: 0")
                all_pred_vals.append(pred_df.values)

            final_preds = pd.DataFrame(np.mean(np.array(all_pred_vals),axis=0), columns=pred_df.columns)
            if dset_to_check == "train":
                final_preds.to_csv("%spredictions%s/%s/MTL/final_train.csv"%(path_to_ext_val_predictions,dpath,dset))
            else:
                final_preds.to_csv("%spredictions%s/%s/MTL/final.csv"%(path_to_ext_val_predictions, dpath,dset))
                
                
        all_pred_vals = []

        ############## MLP averaging #######################
        path_to_preds = "%spredictions%s/%s/MLP_baselines/"%(path_to_ext_val_predictions, dpath,dset)
        print("Saving consensus predictions to %s"%path_to_preds)

        for f in os.listdir(path_to_preds):
            if f.split(".")[0] in [str(i) for i in range(100)]:
                pred_df = pd.read_csv("%s/%s"%(path_to_preds,f), index_col="Unnamed: 0")
                all_pred_vals.append(pred_df.values)

        final_preds = pd.DataFrame(np.mean(np.array(all_pred_vals),axis=0), columns=pred_df.columns)
        final_preds.to_csv(path_to_preds + "final.csv")

Saving consensus test predictions to ../../../md-ad_public_repo_data/External_Validation/predictions/amp-ad_msbb_mssm_affymetrixu133ab_parahippocampal gyrus_rspp-adj.tsv/MTL/200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.001000_[1, 1]_0.100000_20/
Saving consensus predictions to ../../../md-ad_public_repo_data/External_Validation/predictions/amp-ad_msbb_mssm_affymetrixu133ab_parahippocampal gyrus_rspp-adj.tsv/MLP_baselines/
Saving consensus test predictions to ../../../md-ad_public_repo_data/External_Validation/predictions/amp-ad_msbb_mssm_affymetrixu133ab_inferior temporal gyrus_rspp-adj.tsv/MTL/200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.001000_[1, 1]_0.100000_20/


FileNotFoundError: [Errno 2] No such file or directory: '../../../md-ad_public_repo_data/External_Validation/predictions/amp-ad_msbb_mssm_affymetrixu133ab_inferior temporal gyrus_rspp-adj.tsv/MTL/200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.001000_[1, 1]_0.100000_20/'